### **`Libraries needed`**

In [ ]:
import pandas as pd
import numpy as np
import nltk



!pip install demoji
import demoji
demoji.download_codes()

nltk.download('all')


In [ ]:
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import WhitespaceTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.models import *
from keras.layers import *
from keras.callbacks import *
import pandas as pd
import pickle
import numpy as np
import random
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from gensim.models import Word2Vec

### **`Integrating Drive with Colab`**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **`Loading data`**

In [ ]:
col_list = ['Text', 'Label']
data =  pd.read_csv('/content/drive/MyDrive/IR Labelled Dataset/29Nov2020.csv', index_col=0, usecols=col_list)
data

,Label
Text,
Nobody mess with kaurs and bibijis 💪🏾 \n\n#FarmersProtest #kisanandolan https://t.co/uWyYwkaR0Y,NaN
"Bole So Nihal \nSat Shri Akal \n\nIf Jaikara is roared by Sikh regiments while protecting Indian borders then they r patriots, if farmers roar this during protest then they r terrorists.\n\nWhat a double standard?\n#standwithfarmerschallenge⁠￼⁠ #FarmersProtest #FarmerProtest #Sikh https://t.co/OWZrVhAGdX",NaN
"So now what? Are all these separatists too? Oh , let me guess half of north India wants a separate state according to some morons’ logic. #FarmerProtest \n#FarmersProtest \n#standwithfarmerschallenge https://t.co/mqNZ8yV4Nb",NaN
"Who is Barkha Dutt? @BDUTT She calls herself journalist but she worked as a DALAL in political scams. Her name came up in Nira Radia tapes in 175,000 Crore Rs spectrum scandal. Sanghi Kangna @KanganaTeam and Barkha defaming #FarmerProtest https://t.co/vvM31Mthnw https://t.co/tCTlw6nsYZ",NaN
Brampton Protest in support of farmers. #farmerprotest #farmersprotest #StandWithFarmerChallange https://t.co/2LhWydTOA8,NaN
...,...
"Punjabi youngsters clean road at Delhi Border\n\n""We don't want people of Haryana and Delhi to say that Punjabis came and made a mess here"". ""\n#FarmersDelhiProtest \n#FarmersProtest \n#FarmersProtests https://t.co/mDNnEx8lAa",Neutral
FARMERS ARE NOT TERRORISTS\nNO FARMER🚜NO FOOD🌾\n#KisanProtest\n#Standwithfarmerschallenge \n#FarmersProtest https://t.co/OvFC0ipUtC,NaN
"Sikh Women have always been at the forefront of our revolutions and struggles.... this is no different, however today this is just so inspiring #IStandWithFarmers #FarmersBill2020 #FarmersProtest #Equality https://t.co/NYpx7moTLV",NaN


In [ ]:
# Copying data
newData = data.copy

In [ ]:
# Dropping rows with NA value
data = data.dropna()

In [ ]:
data.head()

,Label
Text,
#India: A young farmer turns off a police water cannon &amp; is charged with 'attempted murder'.\n\nThe #farmersprotest are against new laws that will reduce earnings &amp; give more power to corporations \n\n1. https://t.co/POGRjVpQ8L\n\n2. https://t.co/UIDo2tQF3e\n\n https://t.co/jdNHWgEO2i,Anti govt
"True Example of ""Aapdaa mein Avasar"".\n\nA Poor Farmer Doing Bowling Practice in Nets, taking out some time from #FarmersProtest. https://t.co/yucn8tUmy6",Neutral
"I come from a bloodline of farmers and warriors. Where we are named after princesses and earthly kings. Love is our maiden tongue, yet trauma plagues us still. ♥️ I am a daughter of Punjab.\n#FarmersBill2020 #FarmersProtest https://t.co/u6XCKpB0HR",Pro farmer
@SonuSood Sir i want to join #FarmersProtest can you please help me in reaching Delhi any time after 10th of next month.\n\nYou have helped #MigrantWorkers kindly please help me( 8 months jobless girl)\n🙏🙏🙏,Pro farmer
"Godi Media go back, go back! #FarmerProtest https://t.co/SIJZNIyI1l",Anti govt


### **`Pre Processing of data`**

In [ ]:
#======================================================================================================
                       
                                       #PreProcessing

#==============================================================================================
def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon(In Most Cases Working) """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

def doPreProcess(text):

   # ----------------------[1.    Demoji]----------------------------------------------
  text=demoji.replace_with_desc(text, sep="")# Repalce with text


  #------------------------[2.   # and @ handling here]--------------------------------
  #Eiether-------------------------------------------
    # text=re.sub("([@#][A-Za-z0-9]+)","",text)#Replace by space those words with # and @

      #Or
  text=re.sub("([@#])","",text)#Replace by space the chars as  # and @
  #-----------------------------------------------------------


  #------------------------[3.    Remove the Links]--------------------------------
  text=re.sub(r"http\S+", "", text)#Remove the Links
 

  #-------------------------[4.   Emoticons Replacement with text]-------------------
  

  tokenizer = RegexpTokenizer(r'\w+')
  words=WhitespaceTokenizer().tokenize(text.lower())#tokenizer.tokenize(text)#Tokenize and casings
  df = pd.read_excel('/content/drive/MyDrive/IR Labelled Dataset/Emoticons_Emojis_Text.xlsx',sheet_name='Sheet2',header=0,converters={'Emoji':str,'Text':str,'Emoticons':str})
  dictOfEmoticons=df.set_index('Emoji')['Text'].to_dict()
  words=[dictOfEmoticons.get(x) if x in dictOfEmoticons.keys() else x for x in words]
  text=' '.join(words)



  #----------------------[5.       Ignore Non Ascii Characters]-----------------------
  encoded_string = text.encode("ascii", "ignore")  #For ASCII Onlys
  text = encoded_string.decode()

  #----------------------[6.       Single Quotes Handling Here Removal]---------------------:
  text=text.translate(str.maketrans("","", "'"))
  # text=text.translate(str.maketrans("","", string.punctuation))


  #---------------------[7.         Tokenizes]----------------------
  # tokenizer = RegexpTokenizer(r'\w+')
  words=WhitespaceTokenizer().tokenize(text)
 
  #---------------------[8.         Lower Case Folding]------------------------------
  # words=tokenizer.tokenize(text.lower())#Tokenize and casings

  #---------------------[9.         Replace Elongated Words]-------------------------------

  words=[replaceElongated(x) if not wordnet.synsets(x) else x for x in  words]





   #---------------------[10. & 11.        Stopwords Removal & Lematize]--------------------------------------------- 
  wnl = WordNetLemmatizer() 
  words=[wnl.lemmatize(word) for word in words]
  words=[word for word in words if word not in stopwords.words('english') ]



  #---------------------[10. & 11.        Stopwords Removal & Lematize]--------------------------------------------- 
  # wnl = WordNetLemmatizer() 
  # words=[wnl.lemmatize(word) for word in words]
  # words=[word for word in words if word not in stopwords.words('english') ]


  

  #finalText= ' '.join(all_words)

  #-------------------[11.                 Pnctations Removal]------------------------------
  #return re.sub(r'[^\w\s]', '',words)
  return words


def preProcessing(df):
  
  df=df[['Text']]['Text'].apply(doPreProcess)
  return df

#======================================================================================================
                       
                                       #Global Data Frames

#==============================================================================================

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/IR Labelled Dataset/29Nov2020.csv")
df['Preprocessed_tweets']=df['Text'].apply(doPreProcess)

['nobody', 'mess', 'with', 'kaurs', 'and', 'bibijis', 'flexed', 'biceps:', 'medium-dark', 'skin', 'tone', 'farmersprotest', 'kisanandolan']
['nobody', 'mess', 'with', 'kaurs', 'and', 'bibijis', 'flexed', 'biceps:', 'medium-dark', 'skin', 'tone', 'farmersprotest', 'kisanandolan']
['nobody', 'mess', 'kaurs', 'bibijis', 'flexed', 'biceps:', 'medium-dark', 'skin', 'tone', 'farmersprotest', 'kisanandolan']
['bole', 'so', 'nihal', 'sat', 'shri', 'akal', 'if', 'jaikara', 'is', 'roared', 'by', 'sikh', 'regiments', 'while', 'protecting', 'indian', 'borders', 'then', 'they', 'r', 'patriots,', 'if', 'farmers', 'roar', 'this', 'during', 'protest', 'then', 'they', 'r', 'terorists.', 'what', 'a', 'double', 'standard?', 'standwithfarmerschalenge', 'farmersprotest', 'farmerprotest', 'sikh']
['bole', 'so', 'nihal', 'sat', 'shri', 'akal', 'if', 'jaikara', 'is', 'roared', 'by', 'sikh', 'regiment', 'while', 'protecting', 'indian', 'border', 'then', 'they', 'r', 'patriots,', 'if', 'farmer', 'roar', 'this',

In [ ]:
# Creating Word2Vec model
model = Word2Vec(df['Preprocessed_tweets'], min_count=1)

In [ ]:
# Vocabulary Built
vocabulary = model.wv.vocab
vocabulary

{'nobody': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48450>,
 'mess': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c488d0>,
 'kaurs': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48890>,
 'bibijis': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c487d0>,
 'flexed': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48590>,
 'biceps:': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48e10>,
 'medium-dark': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48e50>,
 'skin': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48f90>,
 'tone': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c480d0>,
 'farmersprotest': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48c10>,
 'kisanandolan': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c483d0>,
 'bole': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48050>,
 'nihal': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48cd0>,
 'sat': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48810>,
 'shri': <gensim.models.keyedvectors.Vocab at 0x7f8bd5c48490>,
 'akal': <gensim.mo

In [ ]:
# Printing word embedding for a word 
v1 = model.wv['farmer']
v1

array([ 3.5186822e-04, -3.3770949e-03, -1.2424925e-03,  4.8383055e-03,
       -1.2475085e-03,  3.4409150e-05, -4.4117449e-03,  2.9704706e-03,
        5.8001984e-04, -8.4204084e-06,  2.5674843e-03, -2.7645165e-03,
        3.0935993e-03, -2.0503395e-03,  2.8941766e-03, -1.4597222e-03,
        1.9745547e-03,  4.6853162e-03,  4.2537386e-03, -1.1534706e-03,
        1.9469871e-03, -1.2840207e-03,  2.1029380e-03,  1.5323374e-03,
        5.3731641e-03,  4.9482770e-03, -3.0260079e-03, -5.4458976e-03,
        2.5071336e-03,  4.1044448e-03,  4.3874756e-03, -2.7140290e-03,
        3.9465884e-03,  4.1935998e-03, -5.5555842e-04, -2.0061971e-03,
       -5.4684008e-04,  1.8793360e-03,  2.9283660e-03,  3.8286082e-03,
       -5.1879813e-04,  2.6954415e-03,  4.0534320e-03,  1.6266118e-03,
        1.8325183e-03, -4.0439847e-03,  4.6730801e-03, -3.6325930e-03,
       -2.8049583e-03,  3.1108323e-03,  3.8776237e-03, -4.3919524e-03,
        3.7460180e-03,  3.5970591e-04,  2.3822235e-03,  2.0358821e-03,
      

### **`Finding Similar Words`**

In [ ]:
# Testing the model
sim_words = model.wv.most_similar('farmer')
sim_words

[('kisan"', 0.38248205184936523),
 ('want', 0.35445913672447205),
 ('somebody', 0.32899948954582214),
 ('fakenewsmedia', 0.2935163378715515),
 ('expressing', 0.2929976284503937),
 ('hi', 0.2864779233932495),
 ('people', 0.2713751792907715),
 ('khalistani', 0.2696862816810608),
 ('rubbished', 0.26852214336395264),
 ('ha', 0.2680020034313202)]